In [17]:
# !pip install transformers

In [18]:
# !pip install NERDA

In [1]:
import numpy as np
import pandas as pd
import json
import time
import re
from NERDA.models import NERDA
import torch
import pickle
import sklearn
import random
from sklearn.preprocessing import MultiLabelBinarizer

/home/hl973/.local/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# from NERDA.datasets import get_conll_data, download_conll_data 
# download_conll_data()
# training = get_conll_data('train')
# validation = get_conll_data('valid')

In [3]:
# print(training.keys())
# print(training['sentences'])
# print(training['tags'])
# print(np.array(training['sentences']).shape)
# print(len((training['tags'])))

In [4]:
import transformers
from transformers import BertConfig, BertTokenizer, BertModel

model_dir = "bert-base-chinese"
# config = BertConfig.from_pretrained(model_dir)
# tokenizer = BertTokenizer.from_pretrained(model_dir)
# model = BertModel.from_pretrained(model_dir)

In [6]:
# data in result.json is the labeled ming data
# with open('/home/aistudio/result.json') as jf:
#     ming=json.load(jf)
with open("ming_modern_Chinese.json") as jf:
    ming = json.load(jf)

#unpack .json file, wash the data and transfer them into ndarray
# x_data, y_data corresponds to texts and tags
def dataGen(ming):
    person_ids=np.array(list(ming.keys()))
    person_ids.sort()

    #idNum=len(person_ids)

    x_data=[]
    y_data=[]

    indexer=0
    for person_id in person_ids:
        
        char_tag=ming[person_id]['char_tag']
        x_data.append([])
        y_data.append([])
        omit_len=len(person_id)
        for i in range(omit_len+1,len(char_tag)):
            x_data[indexer].append(char_tag[i][0])
            y_data[indexer].append(char_tag[i][1])
         
        indexer=indexer+1

    for i in range(0,len(y_data)):
        for j in range(0,len(y_data[i])):
            old_text=y_data[i][j]
            #convert labels like 'B_date_reign' into 'B-date-reign'
            new_text=old_text.replace("_","-")
            y_data[i][j]=new_text


    return x_data,y_data,person_ids


#construct train,validate and test set
#train_set_rate indicates the proportion of trainning data
#validate_set_rate indicates the proportion of validation data
def splitTrain(x_data,y_data,person_ids,train_set_rate,validate_set_rate):
    x_data=np.array(x_data)
    y_data=np.array(y_data)
    
    temp=np.array([x_data,y_data])
    temp=temp.T
    
    ming_data=pd.DataFrame(temp,index=person_ids,columns=['text_a','label'])

    #To save data as .tsv files, we must separate the characters using \002 marker
    for i in range(0,len(ming_data['text_a'])):
        #print(i)
        ming_data['text_a'][i]='\002'.join(ming_data['text_a'][i])
        ming_data['label'][i]='\002'.join(ming_data['label'][i])
        ##ming_data['text_a'][i]+='\002'
        ##ming_data['label'][i]+='\002'
        #ming_data['text_a'][i]=str(ming_data['text_a'][i])
        #ming_data['label'][i]=str(ming_data['label'][i])
        
        

    np.random.seed(int(time.time()))
    ming_data=ming_data.sample(frac=1.0)
    
    idNum=len(person_ids)
    train_size=int(np.floor(idNum*train_set_rate))
    validate_size=int(np.floor(idNum*validate_set_rate))

    train_set=ming_data[0:train_size]

    validate_set=ming_data[train_size:train_size+validate_size]
    test_set=ming_data[train_size+validate_size:idNum]
    return train_set,validate_set,test_set,ming_data


#Build the first model: 50% of ming as train and 20% of ming as test
x_data,y_data,person_ids=dataGen(ming)
t,v,testing,ming_data=splitTrain(x_data,y_data,person_ids,0.7,0.15)

#generate x,y of train,validate and test
x_train=np.array(t['text_a'])
y_train=np.array(t['label'])
x_validate=np.array(v['text_a'])
y_validate=np.array(v['label'])

x_test=np.array(testing['text_a'])
y_test=np.array(testing['label'])

# We can save them as .csv files if we want
t.to_csv('train.csv')
v.to_csv('validate.csv')

# Acquire the length of training set
train_len=[]
for i in range(0,len(x_train)):
    train_len.append(len(x_train[i]))

max_len=max(train_len)

# The tag list, which specifies how many tags are of interest.
tag_list = tag_list = ["O",
        "B_date_reign", "I_date_reign",
        "B_date_year", "I_date_year",
        "B_office_voa", "I_office_voa",
        "B_office_title", "I_office_title",
        "B_place_placename", "I_place_placename"]

In [7]:
MAX_LENGTH = 512

In [8]:
sets = [x_train, x_validate, y_train, y_validate, x_test, y_test]
final_sets = [] 
for group in sets:
    data_lst = []
    for sent_or_label in group:
        tokenized = sent_or_label.split('\x02')
        if len(tokenized) > MAX_LENGTH:
            tokenized = sent_or_label.split('\x02')[:MAX_LENGTH]
            data_lst.append(tokenized)
        elif (len(tokenized) > 1 or tokenized[0] != ''):
            data_lst.append(tokenized)
        else:
          # Do not include
          pass
    final_sets.append(data_lst)

train_sent, valid_sent, train_labels, valid_labels, test_sent, test_labels = final_sets[0], final_sets[1], final_sets[2], final_sets[3], final_sets[4], final_sets[5]


In [9]:
print(len(x_test))
print(len(x_validate))
print(len(x_train))

1330
1328
6200


In [10]:
training_dict = {'sentences': train_sent, 'tags': train_labels}
validation_dict = {'sentences': valid_sent, 'tags': valid_labels}
test_dict = {'sentences': test_sent, 'tags':test_labels}

In [11]:
len(training_dict['sentences'])

6197

In [12]:
# import pickle

In [13]:
# pickle.dump(test_dict, open('test_dict.pkl', 'wb'))

In [14]:
print(np.array(train_sent).shape)
print((train_sent[0]))
print(len(train_labels))
print(len(train_labels[0]))
print(len(valid_sent))

(6197,)
['名', '鑄', '，', '以', '字', '行', '，', '鄞', '人', '。', '洪', '武', '十', '六', '年', '以', '國', '子', '生', '擢', '監', '察', '御', '史', '，', '劾', '刑', '部', '尚', '書', '開', '濟', '至', '死', '，', '聲', '動', '朝', '野', '。', '擢', '福', '建', '按', '察', '使', '，', '至', '卽', '誅', '贜', '吏', '數', '十', '人', '，', '興', '學', '勤', '士', '，', '撫', '卹', '軍', '民', '，', '卒', '於', '官', '。']
6197
70
1328


In [15]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
'epochs' : 4,
'warmup_steps' : 200,
'train_batch_size': 13,
'learning_rate': 0.0001
}

In [16]:
model = NERDA(
dataset_training = training_dict,
dataset_validation = validation_dict,
tag_scheme = [
        "B-date-reign", "I-date-reign",
        "B-date-year", "I-date-year",
        "B-office-voa", "I-office-voa",
        "B-office-title", "I-office-title",
        "B-place-placename", "I-place-placename"], 
tag_outside = 'O',
transformer = model_dir,
dropout = dropout,
hyperparameters = training_hyperparameters,
max_len = 512
)

Device automatically set to: cpu


In [48]:
model.train()

  0%|          | 0/511 [00:00<?, ?it/s]


 Epoch 1 / 4


 12%|█▏        | 60/511 [29:27<3:41:29, 29.47s/it]


KeyboardInterrupt: 

In [15]:
# path = ''

In [16]:
# # Part of Task1: save the data into tsv
# # t-> train, v-> validate, testing -> test
# # ming_data -> whole dataset, also saved in case it might be of use
# t.to_csv((path + 'train.tsv'),sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)
# v.to_csv((path + 'validate.tsv'),sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)
# testing.to_csv((path + 'testing.tsv'),sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)
# ming_data.to_csv((path + 'dataset.tsv'),sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)

# # predict_data is the Jin biographical data without tags
# predict_data=pd.read_table('test_data.txt')

# # 'content_without_name' consists of characters in the texts
# text=predict_data['content_without_name']

# # Also save it as .tsv files in case
# text.to_csv((path + 'predict.tsv'),encoding='utf_8_sig')

In [17]:
model.load_network_from_file(model_path='NERDA_model_ming.bin')

'Weights for network loaded from NERDA_model_ming.bin'

In [18]:
test_sent[0]

['銅',
 '仁',
 '人',
 '。',
 '崇',
 '禎',
 '初',
 '官',
 '灤',
 '州',
 '知',
 '州',
 '，',
 '二',
 '年',
 '淸',
 '兵',
 '陷',
 '城',
 '，',
 '自',
 '刎',
 '死',
 '，',
 '贈',
 '光',
 '祿',
 '少',
 '卿',
 '，',
 '諡',
 '烈',
 '愍',
 '。']

In [19]:
# model.evaluate_performance(validation_dict)

In [20]:
(x_data[0])

['字',
 '一',
 '菴',
 '，',
 '號',
 '退',
 '翁',
 '，',
 '上',
 '虞',
 '孫',
 '氏',
 '子',
 '。',
 '年',
 '十',
 '三',
 '為',
 '僧',
 '，',
 '居',
 '杭',
 '州',
 '上',
 '天',
 '竺',
 '寺',
 '。',
 '永',
 '樂',
 '中',
 '奉',
 '命',
 '輯',
 '禪',
 '宗',
 '語',
 '錄',
 '，',
 '後',
 '又',
 '輯',
 '法',
 '華',
 '科',
 '注',
 '，',
 '大',
 '明',
 '法',
 '數',
 '等',
 '書',
 '，',
 '屢',
 '授',
 '僧',
 '錄',
 '司',
 '右',
 '闡',
 '敎',
 '，',
 '年',
 '七',
 '十',
 '四',
 '卒',
 '。']

In [21]:
group_1_test = pd.read_csv('group1.csv', dtype={'id': np.int32, 'bio': str})

In [22]:
group_1_test.columns

Index(['id', 'bio'], dtype='object')

In [23]:
import re
re.sub('[^\u4e00-\u9fff]', '', group_1_test.iloc[0]['bio'])

'字禹學曹州人倜儻多權略自為諸生即習武事弘治十五年進士除刑部主事有重囚越獄人莫敢攖九疇挺槊逐得之遂以武健名正德初錄囚南畿忤劉瑾謫陽山知縣瑾敗復故官歷郎中遷肅州兵備副使總督彭澤之賂土魯番也遣哈密都督寫亦虎仙往九疇奮曰彭公受天子命制邊疆不能身當利害何但模棱為乃練卒伍繕營壘常若臨大敵寫亦虎仙果通賊番酋速檀滿速兒犯嘉峪關遊擊芮寧敗死尋復遣斬巴思等以駝馬乞和而陰遺書虎仙及其姻黨阿剌思罕兒失拜煙答等俾內應九疇知賊計執阿剌思罕兒及斬巴思付獄通事毛鑒等守之鑒等故與通欲縱去眾番皆伺隙為變九疇覺之僇鑒等賊失內應遂拔帳走兵部尚書王瓊惡澤並坐九疇失事罪逮系法司獄以失拜煙答系死為罪除其名世宗即位起故官俄進陜西按察使居數月甘肅總兵官李隆嗾部卒毆殺巡撫許銘焚其屍乃擢九疇右僉都御史巡撫甘肅按驗銘事誅隆及亂卒首事者九疇抵鎮言額軍七萬余存者不及半且多老弱請令召募詔可嘉靖三年速檀滿速兒復以二萬余騎圍肅州九疇自甘州晝夜馳入城射賊賊多死已又出兵擊走之其分掠甘州者亦為總兵官姜奭所敗論功進副都御史賚金幣九疇上言番賊敢入犯者以我納其朝貢縱商販使得稔虛實也寫亦虎仙逆謀已露輸貨權門轉蒙寵幸以犯邊之寇為來享之賓邊臣怵利害拱手聽命致內屬番人勾連接引以至於今今即不能如漢武興大宛之師亦當效光武絕西域之計先後入貢未歸者二百人宜安置兩粵其謀逆有跡者加之刑僇則賊內無所恃必不復有侵軼倘更包含隱忍恐河西十五衛所永無息肩之期也事下總制楊一清頗采其議四年春致仕歸初土魯番敗遁都指揮王輔言速檀滿速兒及牙木蘭俱死於炮九疇以聞後二人上表求通貢帝怪且疑而番人先在京師者為蜚語言肅州之圍由九疇激之帝益信會百戶王邦奇訐楊廷和彭澤詞連九疇吏部尚書桂萼等欲緣九疇以傾澤因請許通貢而追治九疇激變狀大學士一清言事已前決帝不聽逮下詔獄刑部尚書胡世寧言於朝曰世寧司刑而殺忠臣寧殺世寧乃上疏為訟冤曰番人變詐妄騰謗讟欲害我謀臣耳夫其畜謀內寇為日已久一旦擁兵深入諸番約內應非九疇先幾奮僇且近遣屬夷卻其營帳遠交瓦刺擾其窟巢使彼內顧而返則肅州孤城豈復能保臣以為文臣之有勇知兵忘身殉國者無如九疇宜番人深忌而欲殺也惟聽部下卒妄報以滿速兒等為已死則其罪有不免耳已法司具獄亦如世寧言帝卒中萼等言謫戍極邊居十年赦還'

In [24]:
group_1_test.iloc[0]['bio']

'字禹學，曹州人。倜儻多權略。自為諸生，即習武事。弘治十五年進士。除刑部主事。有重囚越獄，人莫敢攖，九疇挺槊逐得之，遂以武健名。正德初，錄囚南畿，忤劉瑾，謫陽山知縣。瑾敗，復故官。歷郎中，遷肅州兵備副使。總督彭澤之賂土魯番也，遣哈密都督寫亦虎仙往。九疇奮曰：「彭公受天子命，制邊疆，不能身當利害，何但模棱為！」乃練卒伍，繕營壘，常若臨大敵。寫亦虎仙果通賊。番酋速檀滿速兒犯嘉峪關，遊擊芮寧敗死。尋復遣斬巴思等以駝馬乞和，而陰遺書虎仙及其姻黨阿剌思罕兒、失拜煙答等俾內應。九疇知賊計，執阿剌思罕兒及斬巴思付獄。通事毛鑒等守之。鑒等故與通，欲縱去，眾番皆伺隙為變。九疇覺之，僇鑒等。賊失內應，遂拔帳走。兵部尚書王瓊惡澤，並坐九疇失事罪，逮系法司獄。以失拜煙答系死為罪，除其名。世宗即位，起故官。俄進陜西按察使。居數月，甘肅總兵官李隆嗾部卒毆殺巡撫許銘，焚其屍。乃擢九疇右僉都御史，巡撫甘肅，按驗銘事，誅隆及亂卒首事者。九疇抵鎮，言額軍七萬余，存者不及半，且多老弱，請令召募。詔可。嘉靖三年，速檀滿速兒復以二萬余騎圍肅州。九疇自甘州晝夜馳入城，射賊，賊多死。已，又出兵擊走之。其分掠甘州者，亦為總兵官姜奭所敗。論功，進副都御史，賚金幣。九疇上言：「番賊敢入犯者，以我納其朝貢，縱商販，使得稔虛實也。寫亦虎仙逆謀已露，輸貨權門，轉蒙寵幸，以犯邊之寇，為來享之賓。邊臣怵利害，拱手聽命，致內屬番人勾連接引，以至於今。今即不能如漢武興大宛之師，亦當效光武絕西域之計。先後入貢未歸者二百人，宜安置兩粵，其謀逆有跡者加之刑僇，則賊內無所恃，必不復有侵軼。倘更包含隱忍，恐河西十五衛所，永無息肩之期也。」事下，總制楊一清頗采其議。四年春致仕歸。初，土魯番敗遁，都指揮王輔言速檀滿速兒及牙木蘭俱死於炮，九疇以聞。後二人上表求通貢，帝怪且疑。而番人先在京師者為蜚語，言肅州之圍，由九疇激之，帝益信。會百戶王邦奇訐楊廷和、彭澤，詞連九疇。吏部尚書桂萼等欲緣九疇以傾澤，因請許通貢，而追治九疇激變狀。大學士一清言事已前決。帝不聽，逮下詔獄。刑部尚書胡世寧言於朝曰：「世寧司刑而殺忠臣，寧殺世寧。」乃上疏為訟冤曰：「番人變詐，妄騰謗讟，欲害我謀臣耳。夫其畜謀內寇，為日已久。一旦擁兵深入，諸番約內應，非九疇先幾奮僇，且近遣屬夷卻其營帳，遠交瓦刺擾其窟巢，使彼內顧而返，則肅州孤城豈復能保？臣以為文臣之有勇知兵忘身殉國者，無如九疇

In [25]:
# Prediction each paragraph in the input csv. Export results of prediction into a readable csv format where rows are alternating between original text, word by word, and predictions, tag by tag. 
rows = []
punct_sent = []
punct_predictions = []
for idx in range(len(group_1_test)):
    paragraph = list(group_1_test.iloc[idx]['bio'])
    preds = model.predict([paragraph])
    padded_paragraph = paragraph.copy()
    padded_preds = preds.copy()[0]
    if len(paragraph) < 512:
        padded_paragraph += [0 for i in range(512 - len(paragraph))]
    else:
        padded_paragraph = padded_paragraph[:512]
    if len(preds[0]) < 512:
        padded_preds += [0 for i in range(512 - len(preds[0]))]
        print(len(preds[0]))
    assert len(padded_paragraph) == 512
    assert len(padded_preds) == 512
    punct_sent.append(padded_paragraph)
    punct_predictions.append(padded_preds)
    rows.append(padded_paragraph)
    rows.append(padded_preds)

/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 1067 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


512


/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 1655 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


512


/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 788 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


...

512


/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 1779 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


512


/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 907 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


512


In [26]:
df = pd.DataFrame(rows, columns=['text/prediction']*512)
df.to_csv('group1_results_formatted.csv')

In [27]:
group_1_test[group_1_test['id'] == 1]

,id,bio
0,1,字禹學，曹州人。倜儻多權略。自為諸生，即習武事。弘治十五年進士。除刑部主事。有重囚越獄，人莫...


In [28]:
testzero_tok = group_1_test[group_1_test['id'] == 1]['bio'][0].split("。")
testzero_tok = [list(sent) for sent in testzero if len(sent) != 0]
test_zero = group_1_test[group_1_test['id'] == 1]['bio'][0]

NameError: name 'testzero' is not defined

In [75]:
results_after_tokenizing = model.predict(testzero_tok)

In [90]:
results1 = model.predict([list(test_zero)])

/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 1067 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


In [91]:
results1 == results_after_tokenizing

False

In [29]:
# Try group1.csv with punctuation removed 
rows_no_punct = []
no_punct_sent = []
no_punct_predictions = []
for idx in range(len(group_1_test)):
    raw_text = group_1_test.iloc[idx]['bio']
    no_punct_text = re.sub('[^\u4e00-\u9fff]', '', raw_text)
    paragraph = list(no_punct_text)
    preds = model.predict([paragraph])
    padded_paragraph = paragraph.copy()
    padded_preds = preds.copy()[0]
    if len(paragraph) < 512:
        padded_paragraph += [0 for i in range(512 - len(paragraph))]
    else:
        padded_paragraph = padded_paragraph[:512]
    if len(preds[0]) < 512:
        padded_preds += [0 for i in range(512 - len(preds[0]))]
        print(len(preds[0]))
    assert len(padded_paragraph) == 512
    assert len(padded_preds) == 512
    no_punct_sent.append(padded_paragraph)
    no_punct_predictions.append(padded_preds)
    rows_no_punct.append(padded_paragraph)
    rows_no_punct.append(padded_preds)

/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 907 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


512


/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 1374 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


512


/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 669 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


...

512
512


/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 1488 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


512


/home/hl973/.local/lib/python3.7/site-packages/NERDA/preprocessing.py:76: UserWarning: Sentence #0 length 762 exceeds max_len 512 and has been truncated
  warnings.warn(msg)


512


In [30]:
df = pd.DataFrame(rows_no_punct, columns=['text/prediction']*512)
df.to_csv('group1_no_punct_results_formatted.csv')

In [31]:
regex = re.compile('[\u4e00-\u9fff]')

In [33]:
# Check if the non-punctuated and punctuated predictions are the same by removing predictions of punctuations from punctuated predictions
removed_punct_sent_indices = []
assert len(punct_sent) == len(no_punct_sent)
for j, sent in enumerate(punct_sent):
    indices = [i for i, element in enumerate(sent) if regex.findall(element)] # find all chinese characters excluding punctuation, etc. 
    for i in range(len(no_punct_sent)):
        punct_idx = indices[i]
        if no_punct_sent[j][i] != punct_sent[j][punct_idx]:
            print(no_punct_sent[j][i], i)
            print(punct_sent[j][punct_idx], punct_idx)
            break
        if no_punct_predictions[j][i] != punct_predictions[j][punct_idx]:
            print(no_punct_predictions[j][i])
            print(punct_predictions[j][punct_idx])
            print(no_punct_sent[j][i], i)
            print(punct_sent[j][punct_idx], punct_idx)
            print(no_punct_sent[j])
            print(no_punct_predictions[j])
            print(punct_sent[j])
            print(punct_predictions[j])

O
B-place-placename
曹 3
曹 4
['字', '禹', '學', '曹', '州', '人', '倜', '儻', '多', '權', '略', '自', '為', '諸', '生', '即', '習', '武', '事', '弘', '治', '十', '五', '年', '進', '士', '除', '刑', '部', '主', '事', '有', '重', '囚', '越', '獄', '人', '莫', '敢', '攖', '九', '疇', '挺', '槊', '逐', '得', '之', '遂', '以', '武', '健', '名', '正', '德', '初', '錄', '囚', '南', '畿', '忤', '劉', '瑾', '謫', '陽', '山', '知', '縣', '瑾', '敗', '復', '故', '官', '歷', '郎', '中', '遷', '肅', '州', '兵', '備', '副', '使', '總', '督', '彭', '澤', '之', '賂', '土', '魯', '番', '也', '遣', '哈', '密', '都', '督', '寫', '亦', '虎', '仙', '往', '九', '疇', '奮', '曰', '彭', '公', '受', '天', '子', '命', '制', '邊', '疆', '不', '能', '身', '當', '利', '害', '何', '但', '模', '棱', '為', '乃', '練', '卒', '伍', '繕', '營', '壘', '常', '若', '臨', '大', '敵', '寫', '亦', '虎', '仙', '果', '通', '賊', '番', '酋', '速', '檀', '滿', '速', '兒', '犯', '嘉', '峪', '關', '遊', '擊', '芮', '寧', '敗', '死', '尋', '復', '遣', '斬', '巴', '思', '等', '以', '駝', '馬', '乞', '和', '而', '陰', '遺', '書', '虎', '仙', '及', '其', '姻', '黨', '阿', '剌', '思', '罕', '兒', '失', '拜', '煙', '答', '等', '

Or, using pickle to load test_dict

In [24]:
test_dict = pickle.load(open('test_dict.pkl', 'rb'))

In [ ]:
# assert len(x_data) == len(y_data)
# indices = [i for i in range(len(x_data))]
# random.shuffle(indices)
# y_true = [y_data[i] for i in range(500)]
# x_subset = [x_data[i] for i in range(500)]
# y_pred = model.predict(x_subset)
y_true = test_dict['tags']
y_pred = model.predict(test_dict['sentences'])

In [71]:
mlb = MultiLabelBinarizer()
mlb.fit(y_true)
y_true_binary = mlb.transform(y_true)
y_pred_binary = mlb.transform(y_pred)

In [ ]:
sklearn.metrics.precision_score(y_true_binary, y_pred_binary, average='weighted')

In [ ]:
sklearn.metrics.recall_score(y_true_binary, y_pred_binary, average='weighted' )